In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# 환경 설정
warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'  # 필요한 경우 시각화 라이브러리 추가 시 사용
plt.rcParams['axes.unicode_minus'] = False

In [4]:
def load_data():
    """
    기본 정보, 매출, 고객 데이터를 CSV 파일로부터 로드합니다.
    """
    print(">> 데이터 로드 시작...")
    try:
        df_info = pd.read_csv('./data/df_info99.csv', encoding='utf-8-sig')
        df_sales = pd.read_csv('./data/df_sales99.csv', encoding='utf-8-sig')
        df_customer = pd.read_csv('./data/df_customer99.csv', encoding='utf-8-sig')
        
        print(f" - Info 데이터 크기: {df_info.shape}")
        print(f" - Sales 데이터 크기: {df_sales.shape}")
        print(f" - Customer 데이터 크기: {df_customer.shape}")
        
        return df_info, df_sales, df_customer
    except FileNotFoundError as e:
        print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요. ({e})")
        return None, None, None

In [5]:
def process_customer_data(df_customer):
    """
    고객 데이터 전처리: 무효 데이터 제거, 비율 정규화, 시계열 피벗 및 트렌드 지표 생성
    """
    print("\n>> 고객 데이터 전처리 수행...")
    df = df_customer.copy()

    # 1. 무효 데이터 필터링 (All-Zero Rows 제거)
    demo_cols = [c for c in df.columns if '고객 비중' in c and ('남성' in c or '여성' in c)]
    revisit_cols = ['재방문 고객 비중', '신규 고객 비중']
    type_cols = ['거주 이용 고객 비율', '직장 이용 고객 비율', '유동인구 이용 고객 비율']

    epsilon = 1e-6
    valid_mask = (df[demo_cols].sum(axis=1) > epsilon) | \
                 (df[revisit_cols].sum(axis=1) > epsilon) | \
                 (df[type_cols].sum(axis=1) > epsilon)
    
    df = df[valid_mask].copy()
    print(f" - 유효 데이터 필터링 완료: {len(df_customer)} -> {len(df)} 행")

    # 2. 정규화 (Normalization to 100%)
    for cols in [demo_cols, revisit_cols, type_cols]:
        row_sum = df[cols].sum(axis=1).replace(0, 1)
        df[cols] = df[cols].div(row_sum, axis=0) * 100

    # 3. 시계열 피벗 (Pivot)
    # 기준년월 파싱
    date_extract = df['기준년월'].astype(str).str.extract(r'^(\d{4})-(\d{2})')
    df['연도'] = pd.to_numeric(date_extract[0])
    df['분기'] = (pd.to_numeric(date_extract[1]) - 1) // 3 + 1
    
    # 피벗 테이블 생성
    numeric_cols = demo_cols + revisit_cols + type_cols
    df_grouped = df.groupby(['가맹점구분번호', '연도', '분기'])[numeric_cols].mean()
    df_pivoted = df_grouped.unstack(level=['연도', '분기'])
    
    # 컬럼명 평탄화 (Flattening)
    df_pivoted.columns = [f"{str(y)[2:]}년_{q}분기_{col}" for col, y, q in df_pivoted.columns]
    
    # 결측치 보간 (Forward/Backward Fill)
    df_pivoted.fillna(method='ffill', axis=1, inplace=True)
    df_pivoted.fillna(method='bfill', axis=1, inplace=True)

    # 4. 트렌드 지표 생성 (변화율 계산)
    df_trends = pd.DataFrame(index=df_pivoted.index)
    base_features = list(set([c.split('_')[-1] for c in df_pivoted.columns]))

    for base in base_features:
        try:
            cols_23 = [c for c in df_pivoted.columns if '23년' in c and base in c]
            cols_24 = [c for c in df_pivoted.columns if '24년' in c and base in c]
            cols_24_h1 = [c for c in cols_24 if '1분기' in c or '2분기' in c]
            cols_24_h2 = [c for c in cols_24 if '3분기' in c or '4분기' in c]

            # 장기 변화 (24년 평균 - 23년 평균)
            if cols_23 and cols_24:
                df_trends[f'장기_{base}_변화'] = df_pivoted[cols_24].mean(axis=1) - df_pivoted[cols_23].mean(axis=1)
            
            # 단기 변화 (24년 하반기 - 24년 상반기)
            if cols_24_h1 and cols_24_h2:
                df_trends[f'단기_{base}_변화'] = df_pivoted[cols_24_h2].mean(axis=1) - df_pivoted[cols_24_h1].mean(axis=1)
        except Exception:
            continue

    return df_trends.fillna(0).reset_index()

In [6]:
def process_sales_data(df_sales):
    """
    매출 데이터 전처리: 장기/단기 매출 구간 변화율 생성 및 컬럼 정리
    """
    print("\n>> 매출 데이터 전처리 수행...")
    df = df_sales.copy()

    # 1. 매출 구간 변화율 생성
    q_2023 = [f'매출금액_23년_{i}분기' for i in range(1, 5)]
    q_2024 = [f'매출금액_24년_{i}분기' for i in range(1, 5)]
    q_2024_h1 = q_2024[:2]
    q_2024_h2 = q_2024[2:]

    # 컬럼 존재 여부 확인 후 계산
    if all(c in df.columns for c in q_2023 + q_2024):
        # 장기 변화율 (24년 평균 - 23년 평균)
        df['매출구간_장기_변화율'] = df[q_2024].mean(axis=1) - df[q_2023].mean(axis=1)
        # 단기 변화율 (24년 하반기 - 24년 상반기)
        df['매출구간_단기_변화율'] = df[q_2024_h2].mean(axis=1) - df[q_2024_h1].mean(axis=1)

    # 2. 불필요 컬럼 제거 (원본 시계열 데이터 등)
    drop_keywords = ['매출금액_23년', '매출금액_24년', '24년도_평균', '24년_평균']
    cols_to_drop = [c for c in df.columns if any(k in c for k in drop_keywords)]
    df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    
    print(f" - 파생변수 생성 및 컬럼 정리 완료")
    return df

In [7]:
def merge_and_finalize(df_info, df_sales, df_customer):
    """
    데이터 병합: 공통 가맹점 추출, 메타 정보 결합, 최종 컬럼 정리 및 저장
    """
    print("\n>> 데이터 병합 및 최종 정리 수행...")
    
    # 1. 공통 가맹점 식별 (Intersection)
    common_ids = set(df_info['가맹점구분번호']) & \
                 set(df_sales['가맹점구분번호']) & \
                 set(df_customer['가맹점구분번호'])
    
    common_ids_list = list(common_ids)
    print(f" - 공통 가맹점 수: {len(common_ids_list)}개")

    # 2. 데이터 필터링
    df_info = df_info[df_info['가맹점구분번호'].isin(common_ids_list)].copy()
    df_sales = df_sales[df_sales['가맹점구분번호'].isin(common_ids_list)].copy()
    df_customer = df_customer[df_customer['가맹점구분번호'].isin(common_ids_list)].copy()

    # 3. 기본 정보 정리 및 타겟 변수 생성
    df_info.rename(columns={'가맹점주소': '도로명주소', '가맹점주소_세부': '도로명주소_세부', '대분류': '업종_대분류'}, inplace=True)
    df_info['폐업여부'] = df_info['폐업일'].notna().astype(int)
    
    # 불필요 식별자 제거
    df_info.drop(columns=['가맹점명', '폐업일', '25년기준_운영개월수'], errors='ignore', inplace=True)

    # 4. 최종 병합 (Merge)
    df_merged = pd.merge(df_sales, df_customer, on='가맹점구분번호', how='inner')
    final_df = pd.merge(df_merged, df_info, on='가맹점구분번호', how='inner')

    # 5. 컬럼 재정렬 (최종 산출물 정의)
    final_columns = [
        # [기본 정보]
        '가맹점구분번호', '업종_대분류', '업종', '상권', '도로명주소', '도로명주소_세부', '개설일', '영업기간_구간',
        # [매출 성과]
        '매출금액_성장_모멘텀', '매출금액_안정성_지수', '매출금액_위험_가속도', 
        '매출구간_장기_변화율', '매출구간_단기_변화율',
        # [경쟁/고객 성과]
        '23_24_유니크고객수_변화', '23_24_객단가_변화', '23_24_동일 업종 매출금액 비율_변화',
        '23_24_동일 업종 매출건수 비율_변화', '23_24_동일 업종 내 매출 순위 비율_변화', 
        '23_24_동일 상권 내 매출 순위 비율_변화',
        # [타겟]
        '폐업여부'
    ]
    # 고객 트렌드 컬럼 자동 추가
    trend_cols = [c for c in final_df.columns if '장기_' in c or '단기_' in c]
    final_columns += trend_cols

    # 존재하는 컬럼만 선택
    available_cols = [c for c in final_columns if c in final_df.columns]
    final_df = final_df[available_cols]

    return final_df

In [ ]:
# 전체 파이프라인 실행

# 1. 데이터 로드
df_info, df_sales, df_customer = load_data()

# 2. 고객 데이터 처리
if df_customer is not None:
    df_customer_trends = process_customer_data(df_customer)

# 3. 매출 데이터 처리
if df_sales is not None:
    df_sales_processed = process_sales_data(df_sales)

# 4. 병합 및 저장
if df_info is not None:
    final_df = merge_and_finalize(df_info, df_sales_processed, df_customer_trends)

    # 결과 저장
    print(f"\n>> 최종 데이터 저장 완료")
    print(f" - 최종 데이터 크기: {final_df.shape}")
    final_df.to_csv('./data/final_df.csv', encoding='utf-8-sig', index=False)
    print(" - 파일 경로: ./data/final_df.csv")